# Agents with multiple signals

In the case where agents have multiple signals, we want to see whether it is incentive compatible for "normal people" to report their true signals.

In [1]:
import numpy as np

In [4]:
HEAD = 0
TAIL = 1
FAIR = 0
BIASED = 1
NORMAL = 0
EXPERT = 1

The assumptions made here is that
1. All the agents have the same priors, i.e. p(fair), p(biased), p(head|fair), p(head|biased) are the same through the lens of both "normal people" and "experts".
2. The fraction of experts is known

In [14]:
# tunable parameters
# probability of states
p_fair = 0.5 # FIXME: p(fair)
p_state_prior = np.array([p_fair, 1-p_fair]) # p(fair), p(biased)

# probability of signals given states
p_signal_prior = np.array([[0.5], [1]]) # FIXME: p(head|fair), p(head|biased)
p_signal_prior = np.append(p_signal_prior, [[1-p_signal_prior[FAIR][HEAD]], [1-p_signal_prior[BIASED][HEAD]]], axis=1)

# fraction of agents
p_normal = 0.5 # FIXME: p(normal)
p_agents = np.array([p_normal, 1-p_normal])

In [94]:
# Calculate probabilistic values
def p_signal(i):
    return np.dot(p_state_prior, p_signal_prior[:, i])

# p_signal = [p(head), p(tail)]
p_signal = np.fromfunction(np.vectorize(p_signal), (2,), dtype=int)

def p_state(i, j):
    return (p_signal_prior[j][i]*p_state_prior[j]/p_signal[i])

# p_state = [[p(fair|head), p(biased|head)],[p(fair|tail), p(biased|tail)]]
p_state = np.fromfunction(np.vectorize(p_state), (2, 2), dtype=int)

def first_order_posterior(i, j):
    return np.dot(p_signal_prior[:, j], p_state[i, :])
    
# first order posterior
p_posterior_1st = np.fromfunction(np.vectorize(first_order_posterior), (2, 2), dtype=int)

def second_order_posterior(x, y, z):
    return np.sum(p_signal_prior[:, x] * p_signal_prior[:, y] * p_signal_prior[:, z] * p_state_prior.T) / np.sum(p_signal_prior[:, x] * p_signal_prior[:, y] * p_state_prior.T)

# second order posterior
p_posterior_2nd = np.fromfunction(np.vectorize(second_order_posterior), (2, 2, 2), dtype=int)


[[[0.9 0.1]
  [0.5 0.5]]

 [[0.5 0.5]
  [0.5 0.5]]]


In [95]:
print("p(head), p(tail)")
print(p_signal)

print("p(fair|head), p(biased|head)")
print("p(fair|tail), p(biased|tail)")
print(p_state)

print("p(head|head), p(tail|head)")
print("p(head|tail), p(tail|tail)")
print(p_posterior_1st)

print("p(head|tail,tail), p(tail|tail,tail)")
print("p(head|head,tail), p(tail|head,tail)")
print("p(head|head, head), p(tail|head, head)")
print(p_posterior_2nd)


p(head), p(tail)
[0.75 0.25]
p(fair|head), p(biased|head)
p(fair|tail), p(biased|tail)
[[0.33333333 0.66666667]
 [1.         0.        ]]
p(head|head), p(tail|head)
p(head|tail), p(tail|tail)
[[0.83333333 0.16666667]
 [0.5        0.5       ]]
p(head|tail,tail), p(tail|tail,tail)
p(head|head,tail), p(tail|head,tail)
p(head|head, head), p(tail|head, head)
[[[0.9 0.1]
  [0.5 0.5]]

 [[0.5 0.5]
  [0.5 0.5]]]


Assume the normal agent in this case has observed a HEAD